# Talk to BigQuery

Demonstrate the different ways that we can use Vertex LLM to ask questions about data in BigQuery.

## Key Takeaways
- Data stays goverened by current organizational policies with usual IAM, VPC, etc. applied while combining data processing abilities of BigQuery and power of LLMs.
- We would like to perform sematic search on text data within a dataset, which isn't possible in SQL (e.g. product review, customer feedback)
- For numeric values, it is possible to convert a table to unstructured text in a way that we can ask questions about the dataset in a more expressive way
- We need to limit the amount of information per sentence when translating from structured data, otherwise similarity search will not be effective. In most cases, you will need multiple sentences per row
- We asked the LLM to generate a summary of a few rows in the table, and then use that to seed our mapper
- We have not tried performing analytics or aggregation operation, but it could be an interesting next step

## Updates May 4th, 2023
- Updated Vertex SDK version
- Changed use case #1 to perform live query instead of stored query
- Load index from Drive if available
- Minor change to code block for adding Project ID and Location

In [1]:
# @title Install Python Libraries
!pip install chromadb
!pip install langchain
!pip install google-cloud-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.4 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 25.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.6/236.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 35.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 34.6 MB/s eta 0

In [2]:
# Vertex AI GenAI Studio SDK - Update the aiplatform SDK
!pip install vertexai
!pip install google.cloud.aiplatform -U -q

In [3]:
#@title Define LLM classes and instantiate
import time

from vertexai.preview.language_models import TextEmbeddingModel, TextGenerationModel

from langchain.embeddings.base import Embeddings
from langchain.llms.base import LLM


def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(f'Sleeping {sleep_time:.1f} seconds')
            time.sleep(sleep_time)


class VertexEmbeddings(Embeddings):

    def __init__(self, model, *, requests_per_minute=15):
        self.model = model
        self.requests_per_minute = requests_per_minute

    def embed_documents(self, texts):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

    while docs:
        # Working in batches of 2 because the API apparently won't let
        # us send more than 2 documents per request to get embeddings.
        head, docs = docs[:2], docs[2:]
        chunk = self.model.get_embeddings(head)
        results.extend(chunk)
        next(limiter)

    return [r.values for r in results]

    def embed_query(self, text):
        single_result = self.embed_documents([text])
        return single_result[0]


class VertexLLM(LLM):

    model: TextGenerationModel
    predict_kwargs: dict

    def __init__(self, model, **predict_kwargs):
        super().__init__(model=model, predict_kwargs=predict_kwargs)

    @property
    def _llm_type(self):
        return 'vertex'

    def _call(self, prompt, stop=None):
        result = self.model.predict(prompt, **self.predict_kwargs)
        return str(result)

    @property
    def _identifying_params(self):
        return {}

# NOTE: Use staging to get 100qps max throughput for embedding indexing
# The embedding content is the same as production so you can use staging
# for indexing and production for querying if desired.
#language_models.TextEmbeddingModel._LLM_ENDPOINT_NAME = (
#  'projects/678515165750/locations/us-central1/endpoints/8156038716377268224')

REQUESTS_PER_MINUTE = 15
#REQUESTS_PER_MINUTE = 6000

model = TextGenerationModel.from_pretrained('google/text-bison@001')
llm = VertexLLM(
    model,
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40
)
embedding = VertexEmbeddings(TextEmbeddingModel.from_pretrained("google/textembedding-gecko@001"), requests_per_minute=REQUESTS_PER_MINUTE)

llm

SyntaxError: 'return' outside function (795389586.py, line 42)

In [ ]:
PROJECT_ID = "mg-ce-demos"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [ ]:
# @title Mount Google Drive
from google.colab import drive

drive.mount('/content/drive')

persistent_path = '/content/drive/Shareddrives/AIF LLM Demos/BigQuery'

Mounted at /content/drive


In [ ]:
# @title BigQuery Setup
%load_ext google.cloud.bigquery

from google.cloud import bigquery
from google.colab import data_table

data_table.enable_dataframe_formatter()

## Use Case 1: Summarization
In this use case, the BiqQuery table contains text information that we would like to perform sematic search and summarization on. We picked the BBC News Archive from the BigQuery public dataset. The idea is that we can index the content of this table using the Vertex Embedding API, and then perform similarity search to find relevant articles when given a question.

User ran a BQ query in the GCP console and now wants to use Vertex LLM to answer questions about the dataset.

To speed up indexing, we limited the result dataset to 1000 articles from the `bigquery-public-data.bbc_news.fulltext` table that have the word `euro` in it. We then generated the embeddings from these text and stored them in a vector DB.

In [ ]:
%%bigquery bbc_df --project $PROJECT_ID

SELECT * FROM `bigquery-public-data.bbc_news.fulltext` WHERE category = "business" LIMIT 100

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
import os

from langchain.document_loaders.dataframe import DataFrameLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores.chroma import Chroma

index_path = f'{persistent_path}/bbc'
bbc_index = None
if os.path.isdir(index_path):
  vectorstore = Chroma(embedding_function=embedding, persist_directory=index_path)
  bbc_index = VectorStoreIndexWrapper(vectorstore=vectorstore)
else:
  loader = DataFrameLoader(bbc_df, page_content_column="body")
  bbc_index = VectorstoreIndexCreator(embedding=embedding, vectorstore_kwargs={
    'persist_directory': index_path
  }).from_loaders([loader])

#### We can then summarize relevant articles based on similarity search

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Set up some parameters
SEARCH_RESULT_COUNT = 5
SIMILIARITY_THRESHOLD = 0.7
question = ('As a commodity trader with business operations in the Europe, '
            'I want to know if there are regional events '
            'that are driving commodity prices up?')

# Based on the question, sarch for relevant articles
similar_docs = bbc_index.vectorstore.similarity_search_with_score(question, llm=llm, k=SEARCH_RESULT_COUNT)
filtered_docs = list(filter(lambda doc: doc[1] <= SIMILIARITY_THRESHOLD, similar_docs))

# Merge the resulting articles, chunk them up and ask Vertex LLM to summarize
merged = "\n".join([doc.page_content for doc, score in filtered_docs])
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 1000,
  chunk_overlap  = 200,
  length_function = len,
)
docs = text_splitter.create_documents([merged])
prompt_template = """Write a concise summary of the following text in bullet points:

{text}

CONCISE SUMMARY IN BULLET POINTS:"""
summary_prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="stuff", prompt=summary_prompt)
context = chain.run(docs)
print(context)

#### Source articles with similarity score (lower value is more similar)

In [ ]:
for doc, score in filtered_docs:
  print(f"Title: {doc.metadata['title']}, Score: {score} ")
  print("")

Title: Winter freeze keeps oil above $50, Score: 0.5575730800628662 

Title: Markets fall on weak dollar fears, Score: 0.5894883871078491 



### Use Case 2: Question and Answer
#### Run a query against the [Google Cloud Release Note dataset](https://pantheon.corp.google.com/marketplace/product/bigquery-public-datasets/google_cloud_release_notes?e=13802955&jsmode=O&mods=-autopush_coliseum&project=octo-aif-sandbox)

In [ ]:
%%bigquery release_notes_df --project $PROJECT_ID


SELECT description, product_name, published_at, product_version_name, release_note_type
FROM `bigquery-public-data.google_cloud_release_notes.release_notes`
ORDER BY published_at DESC
LIMIT 1000

Query is running:   0%|          |

Downloading:   0%|          |

Clean up the data for indexing
- Convert datetime data type to string
- Initialize null data to empty string or zeros

In [ ]:
release_notes_df['published_at'] = release_notes_df['published_at'].astype(str)
release_notes_df = release_notes_df.fillna("")

Generate an index of the release notes

In [ ]:
index_path = f'{persistent_path}/release_notes'
release_note_index = None
if os.path.isdir(index_path):
  vectorstore = Chroma(embedding_function=embedding, persist_directory=index_path)
  release_note_index = VectorStoreIndexWrapper(vectorstore=vectorstore)
else:
  df_loader = DataFrameLoader(release_notes_df, page_content_column="description")
  release_note_index = VectorstoreIndexCreator(embedding=embedding, vectorstore_kwargs={
    'persist_directory': index_path
  }).from_loaders([df_loader])

Ask question about the release note data

In [ ]:
from langchain.chains import RetrievalQA, ConversationChain, SimpleSequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

question = "What are the new features in BigQuery?"

retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=release_note_index.vectorstore.as_retriever())
retrieval_chain({"query": question}, return_only_outputs=True)

IndexError: ignored

### Use Case 3: Tabular data with numeric value
We will use the Google DEI dataset for this experiment. In this case, we want to convert structured data into unstructured text so that we can feed it into Vertex LLM for Q&A. Using the table as context, we can ask the Vertex LLM to generate a sentence to describe each row, which we will use for the conversion.

In [ ]:
%%bigquery dei_df --project $PROJECT_ID

SELECT * FROM `bigquery-public-data.google_dei.dar_intersectional_representation`

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
dei_df

,workforce,report_year,gender_us,race_asian,race_black,race_hispanic_latinx,race_native_american,race_white
0,tech,2018,men,0.310,0.015,0.036,0.005,0.443
1,tech,2023,men,0.347,0.029,0.049,0.005,0.334
2,tech,2020,men,0.335,0.019,0.038,0.005,0.389
3,tech,2019,men,0.321,0.016,0.037,0.005,0.417
4,tech,2015,men,0.275,0.013,0.033,0.005,0.515
...,...,...,...,...,...,...,...,...
115,non_tech_old,2014,women,0.133,0.024,0.033,0.010,0.351
116,non_tech_old,2014,men,0.110,0.017,0.030,0.006,0.338
117,non_tech_old,2015,men,0.109,0.021,0.034,0.006,0.336
118,non_tech_old,2021,women,0.138,0.043,0.045,0.006,0.289


Let's see if we can ask Vertex LLM to generate a sentence that describe one of the rows, which we will then use as a seed to generalize the translation process from tabular data to natural language text

In [ ]:
# Limit the size of the table due to context window
dei_table = dei_df.head(5).to_markdown()

prompt = f"""
  Use the following table in markdown format to answer the question at the end.

  {dei_table}

  Generate a sentence for each row to describe the meaning of the values
"""
response = model.predict(prompt)
response

The table shows the percentage of men in the tech workforce in the United States from 2015 to 2023, broken down by race.

In 2015, 27.5% of men in the tech workforce were white, 31% were Asian, 1.5% were black, 3.6% were Hispanic or Latino, 0.5% were Native American, and 38.9% were of other races.

In 2016, 29.1% of men in the tech workforce were white, 

Using the generated sentence as an example, convert the structured table into natural language text with one sentence per row

In [ ]:
import pandas as pd

data = []
for index, rows in dei_df.iterrows():
  total_percent = rows['race_asian'] + rows['race_black'] + rows['race_hispanic_latinx'] + rows['race_native_american'] + rows['race_white']
  text = """
  In year {0}, {1} made up {2}% of the {3} workforce, and the racial makeup of the workforce was {4}% Asian,
  {5}% Black, {6}% Hispanic or Latino, {7}% Native American, and {8}% White.
  """.format(
      rows['report_year'],
      rows['gender_us'],
      round(total_percent, 2)*100,
      rows['workforce'],
      round(rows['race_asian']*100, 2),
      round(rows['race_black']*100, 2),
      round(rows['race_hispanic_latinx']*100, 2),
      round(rows['race_native_american']*100, 2),
      round(rows['race_white']*100, 2))
  data.append(text)

# Put it in a dataframe so we can easily index it
nl_dei_df = pd.DataFrame(data, columns=['text'])

Let's index the sentences

In [ ]:
index_path = f'{persistent_path}/dei'
dei_index = None

if os.path.isdir(index_path):
  vectorstore = Chroma(embedding_function=embedding, persist_directory=index_path)
  dei_index = VectorStoreIndexWrapper(vectorstore=vectorstore)
else:
  df_loader = DataFrameLoader(nl_dei_df, page_content_column="text")
  dei_index = VectorstoreIndexCreator(embedding=embedding, vectorstore_kwargs={
    'persist_directory': index_path
  }).from_loaders([df_loader])

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

def ask_question(question, max_results=5, threshold=0.5):
  # Based on the question, sarch for relevant articles
  similar_docs = dei_index.vectorstore.similarity_search_with_score(question, llm=llm, k=max_results)
  filtered_docs = list(filter(lambda doc: doc[1] <= threshold, similar_docs))
  context = "\n".join([doc.page_content for doc, score in filtered_docs])
  prompt = f"""
  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  {context}

  Question: {question}
  Answer:
  """
  return model.predict(prompt)

Let's ask the LLM to retrieve exact values from the dataset and see if they are correct (hint: correct)

In [ ]:
ask_question('What percentage of the male workers are Asian in 2016')

37.6%

The question is "What percentage of the male workers are Asian in 2016".

The first context says that "In year 2016, men made up 83.0% of the tech_old workforce, and the racial makeup of the workforce was 28.0% Asian". So the percentage of the male workers that are Asian is 28.0%.

The second context says that "In year 2016, men made up 73.0% of the overall workforce, and the racial makeup

Let's see if we can consistenly get the right answer (hint: incorrect)

In [ ]:
ask_question('What is the percentage of Asian male tech worker in 2016')

25.4%

To answer this question, we need to know the percentage of men in the tech workforce and the percentage of Asians in the tech workforce. In 2016, men made up 83.0% of the tech workforce and the racial makeup of the workforce was 28.0% Asian. So the percentage of Asian male tech worker in 2016 is 83.0% * 28.0% = 25.4%.

And here is the actual value from the table

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT race_asian, workforce
FROM `bigquery-public-data.google_dei.dar_intersectional_representation`
WHERE
  report_year = 2016 AND
  gender_us = 'men'

Query is running:   0%|          |

Downloading:   0%|          |

,race_asian,workforce
0,0.281,tech
1,0.229,overall
2,0.106,non_tech
3,0.280,tech_old
4,0.184,leadership
5,0.109,non_tech_old


It was quite fiddly and it's obvious not always giving us the right answer. But that could be because each sentence packs a lot of data!

Let's try to reduce the amount of information within each sentence so that semantic search has a better chance of picking up the right info. For example, instead of encoding all the fields in each row into the same sentence, we serialize each row into multiple sentences (e.g. In 2015, Asian male made up X% of the tech workforce)

In [ ]:
def get_sentence(year, gender, race, workforce, percent):
  return f'In {year}, {race} {gender} made up {percent}% of the {workforce} workforce'

data = []
for index, rows in dei_df.iterrows():
  for race in ['asian', 'black', 'hispanic latinx', 'native american', 'white']:
    # Special case
    key = 'race_' + race.replace(' ', '_')
    # Get the percentage of the workforce in the
    text = get_sentence(
        rows['report_year'],
        rows['gender_us'],
        race,
        rows['workforce'],
        round(rows[key]*100, 2))
    data.append(text)

# Put it in a dataframe so we can easily index it
nl_dei_df_v2 = pd.DataFrame(data, columns=['text'])

index_path = f'{persistent_path}/dei_v2'
dei_index = None

if os.path.isdir(index_path):
  vectorstore = Chroma(embedding_function=embedding, persist_directory=index_path)
  dei_index = VectorStoreIndexWrapper(vectorstore=vectorstore)
else:
  df_loader = DataFrameLoader(nl_dei_df_v2, page_content_column="text")
  dei_index = VectorstoreIndexCreator(embedding=embedding, vectorstore_kwargs={
    'persist_directory': f'{persistent_path}/dei_v2'
  }).from_loaders([df_loader])

In [ ]:
ask_question('What is the percentage of Asian male tech worker in 2016')

17.6%

In [ ]:
ask_question('Are there more black female leaders in 2022?')

yes

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT report_year, race_black FROM `bigquery-public-data.google_dei.dar_intersectional_representation` WHERE gender_us = 'women' AND workforce = 'leadership'

Query is running:   0%|          |

Downloading:   0%|          |

,report_year,race_black
0,2021,0.013
1,2023,0.024
2,2015,0.012
3,2016,0.010
4,2022,0.020
5,2020,0.011
6,2014,0.010
7,2017,0.011
8,2018,0.013
9,2019,0.011


In [ ]:
ask_question('Overall, which ethnicity has the highest representation in the workforce for 2022?')

In [ ]:
%%bigquery --project $project

SELECT * FROM `bigquery-public-data.google_dei.dar_intersectional_representation` WHERE report_year = 2022 AND workforce = 'overall'